In [1]:
import sys
sys.path.append('../')

from model_runner import ModelRunner
from DBPN_copied import DBPN
from custom_image_dataset import CustomImageDataset
from torch.utils.data import DataLoader
from torchvision import transforms
from PIL import Image
import torch
torch.cuda.empty_cache()
from torch import nn
from image_helper import ImageHelper
import image_patches
import os

/Users/pedrovelozo/anaconda3/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
DATASET_ROOT="../datasets/"
# DATASET_NAME = "Flickr2K"
DATASET_NAME = "Set5"

In [4]:
PATCH_SIZE = 128 # 32 * 4
PATCHES_PER_IMAGE_TRAIN = 45
PATCHES_PER_IMAGE_VALIDATION = 12 # 45 // 4

In [7]:
# image_patches.create_valid_image_patches(f"{DATASET_ROOT}/{DATASET_NAME}", (PATCH_SIZE, PATCH_SIZE, 3), f"{DATASET_ROOT}/{DATASET_NAME}_{PATCH_SIZE}_{PATCH_SIZE}_train_number={PATCHES_PER_IMAGE_TRAIN}", PATCHES_PER_IMAGE_TRAIN)
# image_patches.create_valid_image_patches(f"{DATASET_ROOT}/{DATASET_NAME}", (PATCH_SIZE, PATCH_SIZE, 3), f"{DATASET_ROOT}/{DATASET_NAME}_{PATCH_SIZE}_{PATCH_SIZE}_validation_number={PATCHES_PER_IMAGE_VALIDATION}", PATCHES_PER_IMAGE_VALIDATION)

In [8]:
%%time

SCALE_FACTOR=4

transform_x = transforms.Compose([
    transforms.Resize((PATCH_SIZE // SCALE_FACTOR, PATCH_SIZE // SCALE_FACTOR), interpolation=Image.BICUBIC),
])

transform_y = transforms.Compose([
    transforms.CenterCrop((PATCH_SIZE, PATCH_SIZE))
])

# Set up the data loaders
train_data_set = CustomImageDataset(
    img_dir=f"{DATASET_ROOT}/{DATASET_NAME}_{PATCH_SIZE}_{PATCH_SIZE}_train_number={PATCHES_PER_IMAGE_TRAIN}", transform=transform_x, target_transform=transform_y, tensor_size=PATCH_SIZE)
validation_data_set = CustomImageDataset(
    img_dir=f"{DATASET_ROOT}/{DATASET_NAME}_{PATCH_SIZE}_{PATCH_SIZE}_validation_number={PATCHES_PER_IMAGE_VALIDATION}", transform=transform_x, target_transform=transform_y, tensor_size=PATCH_SIZE)

# Report split sizes
print('Training set has {} instances'.format(len(train_data_set)))
print('Validation set has {} instances'.format(len(validation_data_set)))

Creating Dataset based on folder: ../datasets//Set5_128_128_train_number=45


100%|██████████| 5/5 [00:00<00:00, 55.54it/s]


Creating Dataset based on folder: ../datasets//Set5_128_128_train_number=45
Creating Dataset based on folder: ../datasets//Set5_128_128_validation_number=12


100%|██████████| 5/5 [00:00<00:00, 69.35it/s]

Creating Dataset based on folder: ../datasets//Set5_128_128_validation_number=12
Training set has 30 instances
Validation set has 26 instances
CPU times: user 116 ms, sys: 77.5 ms, total: 193 ms
Wall time: 208 ms


In [9]:
%%time

# https://towardsdatascience.com/7-tips-for-squeezing-maximum-performance-from-pytorch-ca4a40951259
pin_memory = False
batch_size = round(1*(2**6))

train_dataloader = DataLoader(train_data_set, batch_size=batch_size, shuffle=True, pin_memory=pin_memory)
validation_dataloader = DataLoader(validation_data_set, batch_size=batch_size, pin_memory=pin_memory)

CPU times: user 1.11 ms, sys: 2.94 ms, total: 4.05 ms
Wall time: 5.12 ms


In [10]:
# The learning rate is initialized to 1e − 4 for all layers and decrease by a factor of 10 for every 5 × 105 iterations for total 106 iterations.
lr = 1e-4
model_betas = (0.9, 0.999)
device = "cuda" if torch.cuda.is_available() else "cpu"

# Total number of epochs
epochs = 10

# model = DBPN().to(device)
model = DBPN(num_channels=3, base_filter=64,  feat = 256, num_stages=7, scale_factor=4).to(device)
model_runner = ModelRunner()

# For optimization, we use Adam with momentum to 0.9 and weight decay to 1e−4.
optimizer = torch.optim.Adam(model.parameters(), lr=lr, betas=model_betas)

# Dynamically adjust the learning rate policy
lr_scheduler_step_size = epochs // 2
lr_scheduler_gamma = 0.1

optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=1e-4)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=lr_scheduler_step_size, gamma=lr_scheduler_gamma)

In [11]:
%%time
model_runner.train(
    model=model,
    train_dataloader=train_dataloader,
    validation_dataloader=validation_dataloader,
    optimizer=optimizer,
    epochs=100,
    scheduler=scheduler,
    loss_fn=nn.MSELoss(),
)


epoch 0
-------------------------------
Training on 30 samples...
Batch size: 64
Number of batches: 1


  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
model_runner.plot_lr